# Experimental Results

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# import required packages
import sys
sys.path.append('../../')

import numpy as np
import pandas as pd

from src.utils.statistic_functions import read_results, compute_aulc_ranks, extract_execution_times
from src.utils.data_functions import investigate_data_set
from src.utils.plot_functions import plot_aulc_ranks, plot_learning_curves, create_latex_table

from matplotlib import rc
#rc('text', usetex=True)
#rc('font', family='serif')
rc('axes', edgecolor="black")

## Setup
Here, you can define the setup of the conducted experiments. In particular, you can change the `data_set_type` whose explanation is given in the cell below.

In [ ]:
data_set_type = 'real-world' # real-world annotators
#data_set_type = 'simulated-o' # simulated annotators having uniform performance values
#data_set_type = 'simulated-y' # simulated annotator having class-dependent performance values
#data_set_type = 'simulated-x' # simulated annotator having class-dependent performance values


# path of results
abs_path = '../../results/{}/csvs'.format(data_set_type)

# parameters of experiments
budget = 0.4
test_ratio = 0.4
n_repeats = 100 # number of repeated experiments

# parameters of mapal
#beta_0_list = ['1.0', '0.1', '0.01', '0.001', '0.0001', '0.00001']
beta_0_list = ['0.0001']
M_prime = '2'
alpha = '1'

# plot setup
fontsize = 13

# file name to store results
dir_ranking = '../../results/{}/ranking-statistics/{}'.format(data_set_type, data_set_type)
dir_learning = '../../results/{}/learning-curves/{}'.format(data_set_type, data_set_type)

## Data Set Overview

In [ ]:
data_sets = pd.read_csv('../../data/data-set-names-{}.csv'.format(data_set_type), index_col='filename')
data_set_df = {'name': [], 'n-features': [], 'n-instances': [], 'n-instances-per-class': [], 'annotation-perfs': []}
filename_list = []
for d in data_sets.iterrows():
    filename = d[0]
    filename_list.append(filename)
    name = d[1][0]
    n_features, n_instances_per_class, annotation_perfs = investigate_data_set(data_set_name=filename)
    data_set_df['name'].append(name)
    data_set_df['n-features'].append(n_features)
    data_set_df['n-instances'].append(np.sum(n_instances_per_class))
    data_set_df['n-instances-per-class'].append(np.array2string(n_instances_per_class, precision=2, separator=','))
    data_set_df['annotation-perfs'].append(np.array2string(annotation_perfs, precision=2, separator=','))
data_set_df = pd.DataFrame(data_set_df, index=filename_list).sort_values(by=['n-instances'])
data_set_df

In [ ]:
print(data_set_df.to_latex().replace('[', '').replace(']', '').replace(',', ', '))

## Learning Curves and Ranking Statistics

In [ ]:
for beta_0 in beta_0_list:
    # setup name of strategies and load results
    print('beta_0: {}'.format(beta_0))
    filename_list_beta = ['mapal-1-{}-{}-{}-entropy'.format(beta_0, M_prime, alpha), 'ceal', 'ie-thresh', 
                          'ie-adj-cost', 'proactive', 'alio', 'random']
    strategy_df_beta = {
                   'name': ['MaPAL', 'CEAL', 'IEThresh', 'IEAdjCost', 'Proactive', 'ALIO', 'Random'],
                   'color': ['g', 'r', 'm', 'grey', 'k', 'b', 'y'],
                   'line': ['-', '-', '-', '-', '-', '-', '-']
                  }
    strategy_df_beta = pd.DataFrame(strategy_df_beta, index=filename_list_beta)
    results_dict_beta = read_results(abs_path=abs_path, data_set_names=data_set_df.index.values, 
                                     strategy_names=strategy_df_beta.index.values, budget=budget, test_ratio=test_ratio)
    
    
    # compute execution times
    execution_times_df = pd.DataFrame(extract_execution_times(dic=results_dict_beta))
    print(execution_times_df.T.to_latex(float_format="%.4f"))
    
    # compute ranking statistics
    data_set_keys = list(results_dict_beta.keys())
    strategy_keys = list(results_dict_beta[data_set_keys[0]].keys())
    data_set_names = data_set_df.loc[data_set_keys]['name']
    strategy_names = strategy_df_beta.loc[strategy_keys]['name']
    aulc_vals, aulc_ranks, test_results = compute_aulc_ranks(dic=results_dict_beta, c=strategy_keys[0])
    aulc_mean = np.mean(aulc_vals, axis=-1)
    aulc_std = np.std(aulc_vals, axis=-1)
    mean_ranks = np.mean(aulc_ranks, axis=1, keepdims=True)
    
    # print aulc mean table
    print(create_latex_table(aulc_mean=aulc_mean, aulc_std=aulc_std, test_results=test_results, 
                             data_set_names=data_set_names, strategy_names=strategy_names))
    
    # create evaluation plots
    filetype = 'pdf'
    plot_learning_curves(results_dict=results_dict_beta, strategy_df=strategy_df_beta, save=True, fontsize=fontsize, 
                         filename='{}-{}'.format(dir_learning, beta_0), filetype=filetype)
    plot_aulc_ranks(aulc_ranks=aulc_ranks, test_results=test_results, xlabels=data_set_names, 
                    ylabels=strategy_names, filename='{}-{}'.format(dir_ranking, beta_0), filetype=filetype)
    plot_aulc_ranks(aulc_ranks=mean_ranks, xlabels=[''], test_results=test_results, ylabels=strategy_names, 
                    plot_type='mean', filename='{}-{}'.format(dir_ranking, beta_0), filetype=filetype)
